In [1]:
import boto3
import os
import json

# Function to retrieve secrets from AWS Secrets Manager
def get_secret(secret_name):
    client = boto3.client('secretsmanager')
    response = client.get_secret_value(SecretId=secret_name)
    secret = json.loads(response['SecretString'])
    return secret

# Retrieve secrets
secret_name = "S3InputBucket-RAG"  # Replace with your secret name
secrets = get_secret(secret_name)

# Extract secrets
bucket_name = secrets['bucket_name']
role_arn = secrets['role_arn']

# Set the local folder path
local_folder = 'replace with localfolder path'

# Create an STS client
sts_client = boto3.client('sts')

# Assume the role
response = sts_client.assume_role(RoleArn=role_arn, RoleSessionName='AssumeRoleSession')
credentials = response['Credentials']

# Configure AWS SDK with temporary credentials
s3_client = boto3.client('s3',
                         aws_access_key_id=credentials['AccessKeyId'],
                         aws_secret_access_key=credentials['SecretAccessKey'],
                         aws_session_token=credentials['SessionToken'])

def upload_folder_to_s3(local_folder, bucket_name, s3_client):
    for root, dirs, files in os.walk(local_folder):
        # Skip the '.ipynb_checkpoints' directory
        if '.ipynb_checkpoints' in dirs:
            dirs.remove('.ipynb_checkpoints')
        
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, local_folder)
            s3_path = relative_path.replace("\\", "/")  # Ensure S3 path uses forward slashes
            try:
                s3_client.upload_file(local_path, bucket_name, s3_path)
                print(f'Successfully uploaded {local_path} to s3://{bucket_name}/{s3_path}')
            except Exception as e:
                print(f'Failed to upload {local_path} to s3://{bucket_name}/{s3_path}: {e}')

# Call the function to upload the folder
upload_folder_to_s3(local_folder, bucket_name, s3_client)


Successfully uploaded NVD_CVE_7-30_cleaned/CVE-2024-30084.json to s3://rag-llm-docs/CVE-2024-30084.json
Successfully uploaded NVD_CVE_7-30_cleaned/CVE-2024-21417.json to s3://rag-llm-docs/CVE-2024-21417.json
Successfully uploaded NVD_CVE_7-30_cleaned/CVE-2023-35952.json to s3://rag-llm-docs/CVE-2023-35952.json
Successfully uploaded NVD_CVE_7-30_cleaned/CVE-2023-49593.json to s3://rag-llm-docs/CVE-2023-49593.json
Successfully uploaded NVD_CVE_7-30_cleaned/CVE-2024-25575.json to s3://rag-llm-docs/CVE-2024-25575.json
Successfully uploaded NVD_CVE_7-30_cleaned/CVE-2024-35778.json to s3://rag-llm-docs/CVE-2024-35778.json
Successfully uploaded NVD_CVE_7-30_cleaned/CVE-2022-48834.json to s3://rag-llm-docs/CVE-2022-48834.json
Successfully uploaded NVD_CVE_7-30_cleaned/CVE-2024-6338.json to s3://rag-llm-docs/CVE-2024-6338.json
Successfully uploaded NVD_CVE_7-30_cleaned/CVE-2024-30042.json to s3://rag-llm-docs/CVE-2024-30042.json
Successfully uploaded NVD_CVE_7-30_cleaned/CVE-2024-30009.json to 